In [1]:
from bokeh.embed import json_item
from bokeh.layouts import column, row, WidgetBox, layout
from bokeh.palettes import Spectral4, Spectral6, Spectral8
from bokeh.io import output_notebook, output_file, save
from bokeh.plotting import figure, show, save
from bokeh.models import ColumnDataSource, CustomJS, NumeralTickFormatter, Span, Label, HoverTool, Panel, Tabs, CheckboxButtonGroup, CheckboxGroup, Legend
from bokeh.models.widgets import Dropdown, Select, RadioButtonGroup
import pandas as pd
import os

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
curr_path = os.path.abspath(os.path.dirname('file'))
csv_path = os.path.join(curr_path, 'rebate_amounts.csv')

df = pd.read_csv(csv_path)

In [4]:
plot_js_status = """
     
    if (status.active == 0) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
        obj5.visible = true;
        obj6.visible = true;
        obj7.visible = true;
        obj8.visible = true;
        
    }
    
    else if (status.active == 1) {
        obj1.visible = true;
        obj2.visible = true;
        obj3.visible = true;
        obj4.visible = true;
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = false;
        obj8.visible = false;
        
    }

    """

In [5]:
src = ColumnDataSource(df)

def make_plot(src, y1, y2, y3, y4, y5, y6, y7, y8):
    tools = ['save']
    p = figure(plot_width=700, plot_height=500, x_range=(0, 250000), y_range=(0, 7700), tools=tools, title="Coronavirus rebate by family structure")
    p.yaxis.axis_label = "Rebate Amount"
    p.yaxis.formatter = NumeralTickFormatter(format="$0,000")
    p.xaxis.axis_label = "Adjusted Gross Income"
    p.xaxis.formatter = NumeralTickFormatter(format="$0,000")
    
    m3c = p.line(x="AGI", y=y1, color='#3288bd', muted_alpha=0.1, line_width=3, legend_label="Three kids", source=src)
    m2c = p.line(x="AGI", y=y2, color='#fdcc3f', muted_alpha=0.1, line_width=3, legend_label="Two kids", source=src)
    m1c = p.line(x="AGI", y=y3, color='#fc8d59', muted_alpha=0.1, line_width=3, legend_label="One kid", source=src)
    m0c = p.line(x="AGI", y=y4, color='#99d594', muted_alpha=0.1, line_width=3, legend_label="No kids", source=src)
    s3c = p.line(x="AGI", y=y5, color='#3288bd', muted_alpha=0.1, line_width=3, source=src)
    s2c = p.line(x="AGI", y=y6, color='#fdcc3f', muted_alpha=0.1, line_width=3, source=src)
    s1c = p.line(x="AGI", y=y7, color='#fc8d59', muted_alpha=0.1, line_width=3, source=src)
    s0c = p.line(x="AGI", y=y8, color='#99d594', muted_alpha=0.1, line_width=3, source=src)
        
    m3c.visible = False
    m2c.visible = False
    m1c.visible = False
    m0c.visible = False
    
    hover = HoverTool(tooltips=[('AGI: ', '@AGI{$0,0 a}'),
                            ('Rebate: ','$y{$0,0.0 a}')])
    p.add_tools(hover)
    
    label = Label(x=195000, y=70, text='@kpomerleau', background_fill_color='white', render_mode='css')
    p.add_layout(label)
    
    button_status = RadioButtonGroup(labels=["Single", "Married"], active=0)

    callback = CustomJS(code=plot_js_status, args={"p": p, "status": button_status, "obj1": m3c, 
                                            "obj2": m2c, "obj3": m1c, "obj4": m0c, "obj5": s3c,
                                            "obj6": s2c, "obj7": s1c, "obj8": s0c})

    button_status.js_on_change('active', callback)
        
#     widgets = column(children=[button_status, checks_kids], sizing_mode='stretch_both')
    layout = column(p, button_status)
    
    return layout

sen1 = Panel(child=make_plot(src, "sen1m3c", "sen1m2c", "sen1m1c", "sen1m0c", "sen1s3c", "sen1s2c", "sen1s1c", "sen1s0c"), title="Senate Version 1 (3/19)")
sen2 = Panel(child=make_plot(src, "sen2m3c", "sen2m2c", "sen2m1c", "sen2m0c", "sen2s3c", "sen2s2c", "sen2s1c", "sen2s0c"), title="Recovery Rebate")
house = Panel(child=make_plot(src, "housem3c", "housem2c", "housem1c", "housem0c", "houses3c", "houses2c", "houses1c", "houses0c"), title="House Proposal (3/23)")

tabs = Tabs(tabs=[sen2, house, sen1])

show(tabs)
# output_file('rebate_widgets.html', mode='inline')
# save(tabs)

In [ ]:
src = ColumnDataSource(df)
tools = ['save']

p = figure(plot_width=600, plot_height=500, x_range=(0, 225000), y_range=(0, 5000), tools=tools, title="Rebate by Proposal")
p.yaxis.axis_label = "Rebate Amount"
p.yaxis.formatter = NumeralTickFormatter(format="$0,000")
p.xaxis.axis_label = "Adjusted Gross Income"
p.xaxis.formatter = NumeralTickFormatter(format="$0,000")

sen1s0c = p.line(x="AGI", y="sen1s0c", color='#2b83ba', line_width=2, legend_label="Senate Version 1", source=src)
sen2s0c = p.line(x="AGI", y="sen2s0c", color='#fdae61', line_width=2, legend_label="Senate Version 2", source=src)
houses0c = p.line(x="AGI", y="houses0c", color='#d7191c', line_width=2, legend_label="House Proposal", source=src)

sen1s1c = p.line(x="AGI", y="sen1s1c", color='#2b83ba', line_width=2, legend_label="Senate Version 1", source=src)
sen2s1c = p.line(x="AGI", y="sen2s1c", color='#fdae61', line_width=2, legend_label="Senate Version 2", source=src)
houses1c = p.line(x="AGI", y="houses1c", color='#d7191c', line_width=2, legend_label="House Proposal", source=src)

sen1m0c = p.line(x="AGI", y="sen1m0c", color='#2b83ba', line_width=2, legend_label="Senate Version 1", source=src)
sen2m0c = p.line(x="AGI", y="sen2m0c", color='#fdae61', line_width=2, legend_label="Senate Version 2", source=src)
housem0c = p.line(x="AGI", y="housem0c", color='#d7191c', line_width=2, legend_label="House Proposal", source=src)

sen1m1c = p.line(x="AGI", y="sen1m1c", color='#2b83ba', line_width=2, legend_label="Senate Version 1", source=src)
sen2m1c = p.line(x="AGI", y="sen2m1c", color='#fdae61', line_width=2, legend_label="Senate Version 2", source=src)
housem1c = p.line(x="AGI", y="housem1c", color='#d7191c', line_width=2, legend_label="House Proposal", source=src)

sen1s1c.visible = False
sen2s1c.visible = False
houses1c.visible = False
sen1m0c.visible = False
sen2m0c.visible = False
housem0c.visible = False
sen1m1c.visible = False
sen2m1c.visible = False
housem1c.visible = False

select = RadioButtonGroup(labels=["Single, no kids", "Single, one kid", "Married, no kids", "Married, one kid"], active=0)

callback = CustomJS(code=plot_js2, args={"p": p, "select": select, "obj1": sen1s0c, "obj2": sen2s0c, "obj3": houses0c, 
                                        "obj4": sen1s1c, "obj5": sen2s1c, "obj6": houses1c, "obj7": sen1m0c,
                                       "obj8": sen2m0c, "obj9": housem0c, "obj10": sen1m1c, "obj11": sen2m1c,
                                       "obj12": housem1c})

select.js_on_change('active', callback)
layout = column(p, select)
# show(layout)
# output_file('rebate_plot_v2.html', mode='inline')
# save(layout)

In [ ]:
plot_js = """
    if (select.active == 0) {
        obj1.visible = true;
        obj2.visible = true;
        obj3.visible = true;
        obj4.visible = true;
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = false;
        obj8.visible = false;
        obj9.visible = false;
        obj10.visible = false;
        obj11.visible = false;
        obj12.visible = false;
    } else if (select.active == 1) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
        obj5.visible = true;
        obj6.visible = true;
        obj7.visible = true;
        obj8.visible = true;
        obj9.visible = false;
        obj10.visible = false;
        obj11.visible = false;
        obj12.visible = false;
    } else if (select.active == 2) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = false;
        obj8.visible = false;
        obj9.visible = true;
        obj10.visible = true;
        obj11.visible = true;
        obj12.visible = true;
    }
    """
plot_js2 = """
    if (select.active == 0) {
        obj1.visible = true;
        obj2.visible = true;
        obj3.visible = true;
        obj4.visible = false;
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = false;
        obj8.visible = false;
        obj9.visible = false;
        obj10.visible = false;
        obj11.visible = false;
        obj12.visible = false;
    } else if (select.active == 1) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = true;
        obj5.visible = true;
        obj6.visible = true;
        obj7.visible = false;
        obj8.visible = false;
        obj9.visible = false;
        obj10.visible = false;
        obj11.visible = false;
        obj12.visible = false;
    } else if (select.active == 2) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = true;
        obj8.visible = true;
        obj9.visible = true;
        obj10.visible = false;
        obj11.visible = false;
        obj12.visible = false;
    } else if (select.active == 3) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = false;
        obj8.visible = false;
        obj9.visible = false;
        obj10.visible = true;
        obj11.visible = true;
        obj12.visible = true;
    }
    """

In [ ]:
Spectral6

In [ ]:
# output_file('rebate_plot_v1.html')

src = ColumnDataSource(df)

def make_plot(src, y1, y2, y3, y4, y5, y6):
    tools = ['save']
    p = figure(plot_width=600, plot_height=500, x_range=(0, 250000), y_range=(0, 6500), tools=tools, title="Coronavirus rebate by family structure")
    p.yaxis.axis_label = "Rebate Amount"
    p.yaxis.formatter = NumeralTickFormatter(format="$0,000")
    p.xaxis.axis_label = "Adjusted Gross Income"
    p.xaxis.formatter = NumeralTickFormatter(format="$0,000")
    
    m2c = p.line(x="AGI", y=y1, color='#3288bd', muted_alpha=0.1, line_width=3, legend_label="Married, two kids", source=src)
    m1c = p.line(x="AGI", y=y2, color='#99d594', muted_alpha=0.1, line_width=3, legend_label="Married, one kid", source=src)
    m0c = p.line(x="AGI", y=y3, color='#7859a6', muted_alpha=0.1, line_width=3, legend_label="Married, no kids", source=src)
    s2c = p.line(x="AGI", y=y4, color='#fdcc3f', muted_alpha=0.1, line_width=3, legend_label="Single, two kids", source=src)
    s1c = p.line(x="AGI", y=y5, color='#fc8d59', muted_alpha=0.1, line_width=3, legend_label="Single, one kid", source=src)
    s0c = p.line(x="AGI", y=y6, color='#d53e4f', muted_alpha=0.1, line_width=3, legend_label="Single, no kids", source=src)
    
    
#     m0c.muted = True
#     s1c.muted = True
#     s0c.muted = True
    
    hover = HoverTool(tooltips=[('AGI: ', '@AGI{$0,0 a}'),
                            ('Rebate: ','$y{$0,0.0 a}')])
    p.add_tools(hover)
    
    p.legend.click_policy="mute"
    
    label = Label(x=195000, y=70, text='@kpomerleau', background_fill_color='white', render_mode='css')
    p.add_layout(label)
    
    return p

sen1 = Panel(child=make_plot(src, "sen1m2c", "sen1m1c", "sen1m0c", "sen1s2c", "sen1s1c", "sen1s0c"), title="Senate Version 1 (3/19)")
sen2 = Panel(child=make_plot(src, "sen2m2c", "sen2m1c", "sen2m0c", "sen2s2c", "sen2s1c", "sen2s0c"), title="Senate Version 2 (3/21)")
house = Panel(child=make_plot(src, "housem2c", "housem1c", "housem0c", "houses2c", "houses1c", "houses0c"), title="House Proposal (3/23)")

tabs = Tabs(tabs=[house, sen2, sen1])

show(tabs)
# output_file('rebate_plot.html', mode='inline')
# save(tabs)

In [ ]:
p = make_plot(src, "sen1m2c", "sen1m1c", "sen1m0c", "sen1s2c", "sen1s1c", "sen1s0c")

for item in p.legend[0].items:
    print(item.label) # possibly a dict like: {'value': 'foo'}
    print(item.renderers) # a list of GlyphRenderer objects

p.legend.items


In [ ]:
Spectral8

In [ ]:
src = ColumnDataSource(df)

def make_plot(src, y1, y2, y3, y4, y5, y6, y7, y8):
    tools = ['save']
    p = figure(plot_width=700, plot_height=500, x_range=(0, 250000), y_range=(0, 7700), tools=tools, title="Coronavirus rebate by family structure")
    p.yaxis.axis_label = "Rebate Amount"
    p.yaxis.formatter = NumeralTickFormatter(format="$0,000")
    p.xaxis.axis_label = "Adjusted Gross Income"
    p.xaxis.formatter = NumeralTickFormatter(format="$0,000")
    
    m3c = p.line(x="AGI", y=y1, color='#3288bd', muted_alpha=0.1, line_width=3, legend_label="Married, three kids", source=src)
    m2c = p.line(x="AGI", y=y2, color='#66c2a5', muted_alpha=0.1, line_width=3, legend_label="Married, two kids", source=src)
    m1c = p.line(x="AGI", y=y3, color='#abdda4', muted_alpha=0.1, line_width=3, legend_label="Married, one kid", source=src)
    m0c = p.line(x="AGI", y=y4, color='#a894d1', muted_alpha=0.1, line_width=3, legend_label="Married, no kids", source=src)
    s3c = p.line(x="AGI", y=y5, color='#fee08b', muted_alpha=0.1, line_width=3, legend_label="Single, three kids", source=src)
    s2c = p.line(x="AGI", y=y6, color='#fdae61', muted_alpha=0.1, line_width=3, legend_label="Single, two kids", source=src)
    s1c = p.line(x="AGI", y=y7, color='#f46d43', muted_alpha=0.1, line_width=3, legend_label="Single, one kid", source=src)
    s0c = p.line(x="AGI", y=y8, color='#d53e4f', muted_alpha=0.1, line_width=3, legend_label="Single, no kids", source=src)
            
    m3c.visible = False
    m2c.visible = False
    m1c.visible = False
    m0c.visible = False
    s3c.visible = False
    s2c.visible = False
    s1c.visible = False
    s0c.visible = False
    
    legend = Legend(items=[
    ('', [])], location="center")
    p.add_layout(legend, 'right')
    hover = HoverTool(tooltips=[('AGI: ', '@AGI{$0,0 a}'),
                            ('Rebate: ','$y{$0,0.0 a}')])
    p.add_tools(hover)
    
    label = Label(x=195000, y=70, text='@kpomerleau', background_fill_color='white', render_mode='css')
    p.add_layout(label)
    
    button_status = CheckboxButtonGroup(labels=["Single", "Married"], active=[])
    checks_kids = CheckboxButtonGroup(labels=["0 kids", "1 kid", "2 kids", "3 kids"], active=[0,1,2,3])

    callback = CustomJS(code=plot_js, args={"p": p, "status": button_status, "kids": checks_kids, "obj1": m3c, 
                                            "obj2": m2c, "obj3": m1c, "obj4": m0c, "obj5": s3c,
                                            "obj6": s2c, "obj7": s1c, "obj8": s0c, "legend": p.legend})

    button_status.js_on_change('active', callback)
    checks_kids.js_on_change('active', callback)
    
#     p.legend.click_policy = 'hide'
    p.legend.inactive_fill_alpha = 0.9
    p.legend.inactive_fill_color = 'white'
        
    widgets = column(children=[button_status, checks_kids], sizing_mode='stretch_both')
    layout = column(p, widgets)
    
    return layout

sen1 = Panel(child=make_plot(src, "sen1m3c", "sen1m2c", "sen1m1c", "sen1m0c", "sen1s3c", "sen1s2c", "sen1s1c", "sen1s0c"), title="Senate Version 1 (3/19)")
sen2 = Panel(child=make_plot(src, "sen2m3c", "sen2m2c", "sen2m1c", "sen2m0c", "sen2s3c", "sen2s2c", "sen2s1c", "sen2s0c"), title="Senate Version 2 (3/21)")
house = Panel(child=make_plot(src, "housem3c", "housem2c", "housem1c", "housem0c", "houses3c", "houses2c", "houses1c", "houses0c"), title="House Proposal (3/23)")

tabs = Tabs(tabs=[house, sen2, sen1])

# show(tabs)
output_file('rebate_widgets.html', mode='inline')
save(tabs)

In [ ]:
plot_js = """
     
    if (status.active.includes(0)) {
        obj5.visible = true;
        obj6.visible = true;
        obj7.visible = true;
        obj8.visible = true;
        
    }
    
    if (status.active.includes(1)) {
        obj1.visible = true;
        obj2.visible = true;
        obj3.visible = true;
        obj4.visible = true;
    }
    
    if (kids.active.includes(0) == false) {
        obj4.visible = false;
        obj8.visible = false;

    } else if (kids.active.includes(0)) {
        obj4.visible = true;
        obj8.visible = true;
    }
    
    if (kids.active.includes(1) == false) {
        obj3.visible = false;
        obj7.visible = false;
    } else if (kids.active.includes(1)) {
        obj3.visible = true;
        obj7.visible = true;
    }
    
    if (kids.active.includes(2) == false) {
        obj2.visible = false;
        obj6.visible = false;
    } else if (kids.active.includes(2)) {
        obj2.visible = true;
        obj6.visible = true;
    }
    
    if (kids.active.includes(3) == false) {
        obj1.visible = false;
        obj5.visible = false;
    } else if (kids.active.includes(3)) {
        obj1.visible = true;
        obj5.visible = true;
    }
    
    if (status.active.includes(0) == false) {
        obj5.visible = false;
        obj6.visible = false;
        obj7.visible = false;
        obj8.visible = false;
    }
    
    if (status.active.includes(1) == false) {
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
    }

    """